In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [2]:
# import os
# from PIL import Image
# import numpy as np
# import re
# import pandas as pd
# import pickle as pkl

In [ ]:
# def load_pickle(file_path):
#     with open(file_path,'rb') as f:
#         object=pkl.load(f)
#     return object

# CNN arch:

#### Conv block (2-D data)
* Conv2d
* pooling

* Flatten 2D -> 1D

#### Fully Connected block (1-D data)
* Dense

In [ ]:
# class Skin_disease_predictor:
#     def build_model(self):
#         self.model = Sequential()
#         self.model.add(Conv2D(filters = 32, kernel_size = 7, activation = "relu", padding = "same"))
#         self.model.add(MaxPooling2D(pool_size = 2, strides = 2))

#         self.model.add(Conv2D(filters = 64, kernel_size = 5, activation = "relu", padding = "same"))
#         self.model.add(MaxPooling2D(pool_size = 2, strides = 2))

#         self.model.add(Conv2D(filters = 128, kernel_size = 5, activation = "relu", padding = "same"))
#         self.model.add(MaxPooling2D(pool_size = 2, strides = 2))

#         self.model.add(Conv2D(filters = 128, kernel_size = 3, activation = "relu", padding = "same"))
#         self.model.add(MaxPooling2D(pool_size = 2, strides = 2))

#         self.model.add(Conv2D(filters = 64, kernel_size = 3, activation = "relu", padding = "same"))
#         self.model.add(MaxPooling2D(pool_size = 2, strides = 2))

#         self.model.add(Flatten())
#         self.model.add(Dense(units = 4096, activation = "relu"))
#         self.model.add(Dense(units = 1024, activation = "relu"))
#         self.model.add(Dense(units = 256, activation = "relu"))
#         self.model.add(Dense(units = 23, activation = "softmax"))

#         self.model.build(input_shape = (None, 450, 720, 3))
#         self.model.summary()

# model = Skin_disease_predictor()
# model.build_model()

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class SkinDiseasePredictor(nn.Module):
#     def _init_(self):
#         super(SkinDiseasePredictor, self)._init_()
        
#         # Convolutional layers
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, padding=3)  # same padding
#         self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding=2)  # same padding
#         self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2)  # same padding
#         self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)  # same padding
#         self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1)  # same padding
        
#         # Max pooling layer
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         # Fully connected layers
#         self.fc1 = nn.Linear(64 * 28 * 45, 4096)  # Adjusted input size after conv and pooling layers
#         self.fc2 = nn.Linear(4096, 1024)
#         self.fc3 = nn.Linear(1024, 256)
#         self.fc4 = nn.Linear(256, 23)  # 23 output classes
        
#     def forward(self, x):
#         # Convolutional layers with ReLU and pooling
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = self.pool(F.relu(self.conv4(x)))
#         x = self.pool(F.relu(self.conv5(x)))
        
#         # Flatten the output from the convolutional layers
#         x = x.view(-1, 64 * 28 * 45)
        
#         # Fully connected layers
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = F.relu(self.fc3(x))
#         x = self.fc4(x)  # No activation here since softmax is handled in the loss function (CrossEntropyLoss)
        
#         return x

# # Instantiate the model
# model = SkinDiseasePredictor()

# # Print the model summary
# print(model)

In [ ]:
# from torchsummary import summary

# # Assuming your model is defined as model
# model = SkinDiseasePredictor()  # Instantiate your model

# # Move the model to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# # Print model summary
# summary(model, input_size=(3, 224, 224))  # Adjust input size as per your data (e.g., 224x224 for RGB images)

In [1]:
# Block 1: Import necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from PIL import Image
import numpy as np
import re
import pandas as pd
import pickle as pkl
from sklearn.preprocessing import OneHotEncoder
import torch.optim as optim
from torch.utils.data import DataLoader

In [2]:
# Block 2: Load pickle function
def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        obj = pkl.load(f)
    return obj


In [12]:
# Block 3: Define the model class with improvements
class SkinDiseasePredictor(nn.Module):
    def __init__(self):
        super(SkinDiseasePredictor, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=7)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same')
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same')
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding='same')
        self.bn5 = nn.BatchNorm2d(128)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 13 * 22, 4096)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 256)
        self.fc4 = nn.Linear(256, 23)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = self.pool5(F.relu(self.bn5(self.conv5(x))))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x

In [14]:
# Training function for backward propagation and calculating accuracy
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Normalize inputs within each batch
            inputs = inputs / 255.0  # Assuming pixel values are between 0-255
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate the loss
            loss = criterion(outputs, labels)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Backward pass
            loss.backward()
            
            # Update the weights
            optimizer.step()
            
            # Accumulate loss
            running_loss += loss.item() * inputs.size(0)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        # Calculate average loss for the epoch
        epoch_loss = running_loss / len(dataloader.dataset)
        accuracy = (correct_predictions / total_predictions) * 100
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.2f}%')

    print('Training complete')

In [4]:
# Block 4: Assign file path and load pickle
file_path = r'D:\Dipto\I.B.M\CODE_FILES\Filtered_data\formatted_image.pkl'  
images = load_pickle(file_path)
labels=load_pickle(r'D:\Dipto\I.B.M\CODE_FILES\Filtered_data\formatted_labels.pkl')

In [5]:
images.shape

(15499, 450, 720, 3)

In [6]:
# Block 5: Initialize and print model summary
model = SkinDiseasePredictor()
print(model)


SkinDiseasePredictor(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [13]:
# Summary of the model
def print_model_summary(model, input_size):
    from torchsummary import summary
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    summary(model, input_size)

# Example usage
if __name__ == "__main__":
    model = SkinDiseasePredictor()
    print_model_summary(model, (3, 450, 720))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 444, 714]           9,472
       BatchNorm2d-2         [-1, 64, 444, 714]             128
         MaxPool2d-3         [-1, 64, 222, 357]               0
            Conv2d-4        [-1, 128, 218, 353]         204,928
       BatchNorm2d-5        [-1, 128, 218, 353]             256
         MaxPool2d-6        [-1, 128, 109, 176]               0
            Conv2d-7        [-1, 256, 109, 176]         819,456
       BatchNorm2d-8        [-1, 256, 109, 176]             512
         MaxPool2d-9          [-1, 256, 54, 88]               0
           Conv2d-10          [-1, 256, 54, 88]         590,080
      BatchNorm2d-11          [-1, 256, 54, 88]             512
        MaxPool2d-12          [-1, 256, 27, 44]               0
           Conv2d-13          [-1, 128, 27, 44]         295,040
      BatchNorm2d-14          [-1, 128,

In [8]:
dt=pd.DataFrame({"labels": labels})

In [9]:
ohe=OneHotEncoder(sparse_output=False)
ohe_labels=ohe.fit_transform(dt)
print(ohe_labels)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [10]:
ohe_labels.shape

(15499, 23)

In [16]:
# Example usage
if __name__ == "__main__":
    # Initialize the model
    model = SkinDiseasePredictor()

    # Load the dataset in smaller batches to avoid memory issues
    batch_size = 1000
    total_samples = 15499
    
    # Loop through the dataset in batches of 1000 to create dataloaders and train the model
    for i in range(0, total_samples, batch_size):
        # Generate a batch of images and labels
        batch_images = np.random.randint(0, 256, (min(batch_size, total_samples - i), 450, 720, 3), dtype=np.uint8)
        batch_labels = np.eye(23)[np.random.randint(0, 23, min(batch_size, total_samples - i))]  # One-hot encoded labels
        
        # Convert numpy arrays to torch tensors
        inputs = torch.tensor(batch_images, dtype=torch.float32).permute(0, 3, 1, 2)  # Convert to shape (batch_size, 3, 450, 720)
        labels = torch.tensor(np.argmax(batch_labels, axis=1), dtype=torch.long)  # Convert one-hot to class indices
        
        # Define your dataset and dataloader
        dataset = torch.utils.data.TensorDataset(inputs, labels)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)

        # Train the model on this batch
        train_model(model, dataloader, criterion, optimizer, num_epochs=10)

    # Print the model summary
    print_model_summary(model, (3, 450, 720))

Epoch [1/10], Loss: 3.1458, Accuracy: 4.00%
Epoch [2/10], Loss: 3.1350, Accuracy: 4.40%
Epoch [3/10], Loss: 3.1362, Accuracy: 5.20%
Epoch [4/10], Loss: 3.1321, Accuracy: 5.50%
Epoch [5/10], Loss: 3.1331, Accuracy: 5.70%
Epoch [6/10], Loss: 3.1381, Accuracy: 5.60%
Epoch [7/10], Loss: 3.0945, Accuracy: 6.90%
Epoch [8/10], Loss: 3.0835, Accuracy: 9.50%
Epoch [9/10], Loss: 3.0775, Accuracy: 11.10%
Epoch [10/10], Loss: 3.0764, Accuracy: 12.00%
Training complete
Epoch [1/10], Loss: 3.1415, Accuracy: 4.50%
Epoch [2/10], Loss: 3.1177, Accuracy: 6.70%
Epoch [3/10], Loss: 3.0936, Accuracy: 9.10%
Epoch [4/10], Loss: 3.0683, Accuracy: 12.60%
Epoch [5/10], Loss: 3.0605, Accuracy: 13.20%
Epoch [6/10], Loss: 3.0526, Accuracy: 13.80%
Epoch [7/10], Loss: 3.0563, Accuracy: 14.00%
Epoch [8/10], Loss: 3.0433, Accuracy: 15.00%
Epoch [9/10], Loss: 3.0402, Accuracy: 15.80%
Epoch [10/10], Loss: 3.0070, Accuracy: 19.10%
Training complete
Epoch [1/10], Loss: 3.1415, Accuracy: 4.30%
Epoch [2/10], Loss: 3.1030, A

In [19]:
# Save the model to a specified path
save_path = r"D:\Dipto\I.B.M\CODE_FILES\skin_disease_predictor.pth"
try:
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')
except IOError:
    print(f'Error: Could not save the model to {save_path}. Please check the path and permissions.')

Model saved to D:\Dipto\I.B.M\CODE_FILES\skin_disease_predictor.pth
